In [1]:
import csv

FAQ_LIST = []
with open("faq_responses.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        keys = [k.strip().lower() for k in row["keywords"].split(";") if k.strip()]
        FAQ_LIST.append({"keywords": keys, "response": row["response"]})

print("✅ Loaded", len(FAQ_LIST), "FAQ entries.")

✅ Loaded 24 FAQ entries.


In [2]:
import time
from datetime import datetime
from IPython.display import display
import ipywidgets as w
import csv
from pathlib import Path

CHAT_HISTORY = []          
MAX_HISTORY = 120

FAQ_LIST = []  

def load_faq_csv(csv_path="faq_responses.csv"):
    p = Path(csv_path)
    if not p.exists():
        raise FileNotFoundError(f"Can't find {csv_path}. Is it in the same folder as the notebook?")
    with p.open(newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            keys = [k.strip().lower() for k in row["keywords"].split(";") if k.strip()]
            FAQ_LIST.append({"keywords": keys, "response": row["response"]})

load_faq_csv("faq_responses.csv")

def simple_bot_reply(message: str):
    msg = message.lower().strip()
    for entry in FAQ_LIST:
        for k in entry["keywords"]:
            if k and k in msg:
                return entry["response"]
    return "Totally get it—tell me a bit more so I can point you to the right feature or plan."

def log_message(role: str, text: str):
    """Append chat messages to a CSV file with timestamp."""
    write_header = not LOG_FILE.exists()
    with LOG_FILE.open("a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(["timestamp", "role", "message"])
        writer.writerow([datetime.now().isoformat(timespec="seconds"), role, text])

css = w.HTML("""
<style>
:root{
  --bg: #0b1220;
  --card: #111827;
  --accent: linear-gradient(90deg,#6366f1,#22d3ee);
  --bot: #ecfeff;
  --botText: #083344;
  --user: #312e81;
  --userText: #eef2ff;
  --border: #1f2937;
  --muted: #94a3b8;
}
body { background: var(--bg); }
.chat-wrap {max-width: 820px; margin: 0 auto; font-family: ui-sans-serif, system-ui, -apple-system; color: #e5e7eb;}
.card {background: var(--card); border: 1px solid var(--border); border-radius: 16px; padding: 10px;}
h2.title { 
  margin: 6px 0 10px; 
  background: var(--accent); 
  -webkit-background-clip: text; 
  background-clip: text; 
  color: transparent; 
  font-weight: 800; 
  letter-spacing: .3px;
}
.msg {padding: 10px 14px; border-radius: 16px; margin: 6px 0; line-height: 1.35; box-shadow: 0 1px 0 rgba(255,255,255,0.05) inset;}
.user {background: var(--user); color: var(--userText); align-self: flex-end; max-width: 80%;}
.bot {background: var(--bot); color: var(--botText); align-self: flex-start; max-width: 80%;}
.meta {font-size: 11px; color: var(--muted); margin-top: 2px;}
.row {display: flex; gap: 8px; align-items: center;}
.bubble-col {display: flex; flex-direction: column; align-items: stretch;}
.controls { display:flex; gap:8px; align-items:center; }
.btn { padding: 8px 12px; border-radius: 12px; border: 1px solid var(--border); cursor: pointer;
       background: #0b1020; color: #e5e7eb; }
.btn:hover { filter: brightness(1.1); }
.status { color: var(--muted); min-height: 16px; }
</style>
""")

header = w.HTML("<h2 class='title'>Live Chat Demo</h2>")
chat_box = w.VBox(layout=w.Layout(height='420px', overflow_y='auto'))
user_input = w.Text(placeholder="Type a message and press Enter…", layout=w.Layout(flex='1'))
send_btn = w.Button(description="Send", tooltip="Send", icon='paper-plane')
reset_btn = w.Button(description="Reset", tooltip="Clear chat", button_style='warning')
status = w.HTML("<span class='status'></span>")

controls = w.HBox([user_input, send_btn, reset_btn], layout=w.Layout(margin='6px 0'))
ui = w.VBox([css, w.HTML("<div class='chat-wrap'>"),
              header,
              w.HTML("<div class='card'>"),
              chat_box,
              controls,
              status,
              w.HTML("</div></div>")])

def render_messages():
    children = []
    for m in CHAT_HISTORY[-MAX_HISTORY:]:
        role = m["role"]; text = m["text"]
        ts = datetime.fromtimestamp(m["ts"]).strftime("%H:%M:%S")
        klass = "user" if role == "user" else "bot"
        bubble = w.HTML(f"""
        <div class="bubble-col">
          <div class="msg {klass}">{text}</div>
          <div class="meta">{role} · {ts}</div>
        </div>
        """)
        row = w.HBox([bubble], layout=w.Layout(justify_content='flex-end' if role=='user' else 'flex-start'))
        children.append(row)
    chat_box.children = children
    status.value = "<span class='status'></span>"

def add_message(role: str, text: str):
    CHAT_HISTORY.append({"role": role, "text": text, "ts": time.time()})
    render_messages()

def handle_send(_=None):
    msg = user_input.value.strip()
    if not msg:
        status.value = "<span class='status' style='color:#fca5a5'>Type something first.</span>"
        return
    user_input.value = ""
    add_message("user", msg)
    status.value = "<span class='status'>…thinking</span>"
    bot_text = simple_bot_reply(msg)
    add_message("bot", bot_text)
    status.value = "<span class='status'></span>"

def handle_reset(_):
    CHAT_HISTORY.clear()
    add_message("bot", "Chat cleared. How can I help?")

user_input.on_submit(handle_send)
send_btn.on_click(handle_send)
reset_btn.on_click(handle_reset)

add_message("bot", "Hey! I’m your friendly demo bot. Ask about pricing, features, integrations, or support.")
display(ui)

/var/folders/y6/dhg3gr295vg6mxbrx7v9_0nm0000gn/T/ipykernel_72047/2241792458.py:134: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  user_input.on_submit(handle_send)
